In [1]:
# IMPORT LIBRARIES

import numpy as np													# Numerical computations
import pandas as pd 												# Data structures
import matplotlib.pyplot as plt										# Statistic visualizations
import matplotlib.ticker as mticker									# Ticker formatting
from pandas.plotting import scatter_matrix							# Scatter matrix
import seaborn as sns												# Statistic visualizations
import statsmodels.api as sm										# Model construction
from scipy import stats			                                    # Statistics
import statsmodels.api as sm                                        # Model construction
from sklearn.linear_model import LinearRegression, Ridge, Lasso, RidgeCV, LassoCV	# Regression model
from sklearn.preprocessing import MinMaxScaler, StandardScaler, OneHotEncoder      # Assigns numeric ranges 0-1
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier					# Decision Trees
from sklearn.model_selection import train_test_split, KFold				# Test models
from sklearn.metrics import root_mean_squared_error, r2_score, accuracy_score		# Model accuracy
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
import xgboost
from sklearn.datasets import load_iris, load_diabetes
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor 
#from utils import pipeline
from xgboost import XGBClassifier, XGBRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold

In [2]:
# Step 2: Load the data
df = pd.read_csv("anon_cust_data_1.csv")

In [3]:
df.head()

,Unnamed: 0,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,0,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,1,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,2,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,3,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,4,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0        7043 non-null   int64  
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


In [5]:
df.describe()

,Unnamed: 0,SeniorCitizen,tenure,MonthlyCharges
count,7043.000000,7043.000000,7043.000000,7043.000000
mean,3521.000000,0.162147,32.371149,64.761692
std,2033.283305,0.368612,24.559481,30.090047
min,0.000000,0.000000,0.000000,18.250000
25%,1760.500000,0.000000,9.000000,35.500000
50%,3521.000000,0.000000,29.000000,70.350000
75%,5281.500000,0.000000,55.000000,89.850000
max,7042.000000,1.000000,72.000000,118.750000


In [6]:
for col in df.columns:
    print(df[col].value_counts())

Unnamed: 0
0       1
4691    1
4702    1
4701    1
4700    1
       ..
2344    1
2343    1
2342    1
2341    1
7042    1
Name: count, Length: 7043, dtype: int64
gender
Male      3555
Female    3488
Name: count, dtype: int64
SeniorCitizen
0    5901
1    1142
Name: count, dtype: int64
Partner
No     3641
Yes    3402
Name: count, dtype: int64
Dependents
No     4933
Yes    2110
Name: count, dtype: int64
tenure
1     613
72    362
2     238
3     200
4     176
     ... 
28     57
39     56
44     51
36     50
0      11
Name: count, Length: 73, dtype: int64
PhoneService
Yes    6361
No      682
Name: count, dtype: int64
MultipleLines
No                  3390
Yes                 2971
No phone service     682
Name: count, dtype: int64
InternetService
Fiber optic    3096
DSL            2421
No             1526
Name: count, dtype: int64
OnlineSecurity
No                     3498
Yes                    2019
No internet service    1526
Name: count, dtype: int64
OnlineBackup
No                     3

In [7]:
(pd.Categorical(df['InternetService']).codes)[73]

np.int8(2)

No phone service
No internet service

In [8]:
# df.loc[df['OnlineSecurity'] == 'No internet service', 'No internet service'] = 'No'
# 

In [9]:
# df.iloc[73]

In [15]:
def change_to_no(dataframe: pd.DataFrame, replace: str, new_string: str) -> pd.DataFrame:
    """Changes any no [insert specific service here] to just no."""
    func_df = dataframe.copy()
    columns = func_df.columns
    # try:
    #     replacer = kwargs['replace']
    # except:
    #     print(f"no 'replace' argument")
    for col in columns:
        if type(df[col][0]) != str:
            print(f"There is no '{replace}' in the {col} column.")
            pass
        else:
            func_df.loc[func_df[col] == replace, col] = new_string
    return func_df

In [16]:
df = change_to_no(df, 'No phone service', 'No')
df = change_to_no(df, 'No internet service', 'No')
df.iloc[73]

There is no 'No phone service' in the Unnamed: 0 column.
There is no 'No phone service' in the SeniorCitizen column.
There is no 'No phone service' in the tenure column.
There is no 'No phone service' in the MonthlyCharges column.
There is no 'No internet service' in the Unnamed: 0 column.
There is no 'No internet service' in the SeniorCitizen column.
There is no 'No internet service' in the tenure column.
There is no 'No internet service' in the MonthlyCharges column.


Unnamed: 0                                 73
gender                                   Male
SeniorCitizen                               0
Partner                                   Yes
Dependents                                Yes
tenure                                     62
PhoneService                              Yes
MultipleLines                             Yes
InternetService                            No
OnlineSecurity                             No
OnlineBackup                               No
DeviceProtection                           No
TechSupport                                No
StreamingTV                                No
StreamingMovies                            No
Contract                             Two year
PaperlessBilling                          Yes
PaymentMethod       Bank transfer (automatic)
MonthlyCharges                          24.25
TotalCharges                           1424.6
Churn                                      No
Name: 73, dtype: object